#### 환경설정

In [1]:
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(r'C:\Users\Eunhoo\Code\NLP\chromedriver.exe')

#### 크롤링할 가수명 저장

In [2]:
singer=  ['룰라','쿨','터보','알이에프']
singer_id =[]

In [3]:
for i in singer:
    driver.get("https://www.melon.com/index.htm")   
    search = driver.find_element_by_xpath('//*[@id="top_search"]') #검색창 xpath
    search.clear() #검색창 clear
    search.send_keys(i+'\n')  #검색창에 가수 입력후 엔터
    
    driver.find_element_by_xpath('//*[@id="conts"]/div[3]/div/div[1]/div/a/strong').click() #검색한 아티스트 클릭
    url = driver.current_url
    id = url.replace('https://www.melon.com/artist/timeline.htm?artistId=', '') #url에 있는 가수 고유번호 저장
    
    singer_id.append(id)

#### 가수별 고유번호 불러오기

In [4]:
print(singer_id)

['106610', '100004', '100059', '100015']


#### 크롤링에 필요한 배열 생성

In [5]:
title_ary=[]
date_ary=[]
genre_ary=[]
lyrics_ary=[]
lyricist_ary=[]
composer_ary=[]
singer_ary=[]

In [8]:
def search(id,n):   # 가수id를 받아 (제목,가수,가사, 발매일,장르,작사,작곡)  n개 만큼탐색  
    #id에 맞는 가수의 곡 페이지(인기순)으로 이동
    driver.get('https://www.melon.com/artist/song.htm?artistId='+str(id)+'#params%5BlistType%5D=A&params%5BorderBy%5D=POPULAR_SONG_LIST&params%5BartistId%5D='+str(id)+'&po=pageObj&startIndex=1')
    driver.implicitly_wait(1)

    for i in range(1,n+1):

        driver.execute_script('window.scrollTo(0,200)')
        
        # i번째 곡정보 클릭
        driver.find_element_by_xpath('//*[@id="frm"]/div/table/tbody/tr['+ str(i)+']/td[3]/div/div/a[1]').click()
        
        # 발매일 먼저 검사
        str_tmp=driver.find_element_by_xpath('//*[@id="downloadfrm"]/div/div/div[2]/div[2]/dl/dd[2]').text
        arr = str_tmp.split('.')
        
        # 발매일이 94년~99년까지인 곡만 정보 수집
        if( int(arr[0]) < 2000 and int(arr[0]) > 1993):
            #print("input: ", str_tmp)
            date_ary.append(str_tmp)
            
            title=driver.find_element_by_xpath('//*[@id="downloadfrm"]/div/div/div[2]/div[1]/div[1]')
            title_ary.append(title.text)
            
            singer=driver.find_element_by_xpath('//*[@id="downloadfrm"]/div/div/div[2]/div[1]/div[2]/a/span[1]')
            singer_ary.append(singer.text)
            
            genre=driver.find_element_by_xpath('//*[@id="downloadfrm"]/div/div/div[2]/div[2]/dl/dd[3]')
            genre_ary.append(genre.text)
            
            try:
                lyricist=driver.find_element_by_xpath('//*[@id="conts"]/div[3]/ul/li[1]/div[2]/div[1]/a')
                lyricist_ary.append(lyricist.text)
            except:
                lyricist_ary.append('NULL')
        
            try:
                composer=driver.find_element_by_xpath('//*[@id="conts"]/div[3]/ul/li[2]/div[2]/div[1]/a')
                composer_ary.append(composer.text)
            except:
                composer_ary.append('NULL')
        
            try:
                lyrics=driver.find_element_by_xpath('//*[@id="d_video_summary"]')
                lyrics_ary.append(lyrics.text.replace('\n',' '))
            except:
                lyrics_ary.append('NULL')
            
            driver.back() #뒤로가기 ->다시 곡 목록으로 이동
            
        else:
            driver.back()

#### 가수당 곡 10개씩 검사 (수집되는 것은 발매일이 94~99년도 사이)

In [9]:
for i in singer_id:
   search(i,10)

In [10]:
import pandas as pd
df=pd.DataFrame({"년도":date_ary,"가수":singer_ary,"제목":title_ary,"장르":genre_ary,
                 "작사가":composer_ary,"작곡가":lyricist_ary,"가사":lyrics_ary})

In [11]:
df

,년도,가수,제목,장르,작사가,작곡가,가사
0,1997.04.01,룰라,3!4!,댄스,이현도,이현도,여기 숨쉬는 이 시간은 나를 어데로 데려갈까 많은 기쁨과 한숨들이 뒤섞인 이곳에서 ...
1,1995,룰라,날개 잃은 천사,댄스,최준영,이건우,아 그럴거야 나를 아낄려고 굳이 내게 말 안하고 멀리 떠나갔던가 아 그럴거야 나를 ...
2,1996.06.15,룰라,3!4!,댄스,이현도,이현도,여기 숨쉬는 이 시간은 나를 어데로 데려갈까 많은 기쁨과 한숨들이 뒤섞인 이 곳에서...
3,1995,룰라,천상유애(天上有愛),댄스,The Season,이건우,저 끝에서 끝까지 목이 메이도록 말하고 말거야 무척 소중한걸 안다고 거리마다 외칠 ...
4,1997,룰라,연인,댄스,이상민,이상민,바보 바보 바보야 바보 바보 바보야 바보 바보 바보야 사랑 앞에서 오늘도 넌 튕겨 ...
5,1995,룰라,프로와 아마츄어,댄스,최준영,룰라,나는 나는 모든걸 할수가 있어 너가 나를 사랑하게 할수도 있고 너를 감동 시킬수도 ...
6,1997.04.01,룰라,비밀은 없어,댄스,박선만,박선만,우우 정말이야 이제 그대에게 비밀은 없어 우우 알고 있지 사랑한단 말은 안해도 처음...
7,1995,룰라,사랑법,댄스,The Season,룰라,하지만 사랑이 자리잡혀 간다면 진실한 사랑이 필요하기 시작한거야 갖기 위해서 해왔던...
8,1999.02,룰라,기도 (Good),댄스,최준영,이상민,Roo'Ra deeper lover live lover live 라라따리 라리라 라...
9,1995.10.01,쿨 (COOL),슬퍼지려 하기 전에,댄스,최준영,최준영,나 가끔씩 그대 생각을 할 때마다 나 가끔 그대 생각을 할때마다 늘 가까운 듯 멀게...


#### 저장되어 있는 아이돌 리스트 csv파일 불러오기

In [13]:
idol_list = pd.read_csv('94to95_idol_list.csv')
idol_list.shape

(4, 7)

In [14]:
idol_list.columns

Index(['artist', 'artist_eng', 'member', 'leader', 'gender', 'agency',
       'idol_id'],
      dtype='object')

#### 성별 column 추가 위한 작업

In [15]:
add_gender = []
add_gender = idol_list.gender
add_gender

0    혼
1    혼
2    남
3    남
Name: gender, dtype: object

In [16]:
# 성별 영어로 바꿔주기

for i in range(len(add_gender)):
    if(add_gender[i] == '남'):
        add_gender[i] = 'male'
    else:
        add_gender[i] = 'mixed'

add_gender

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


0    mixed
1    mixed
2     male
3     male
Name: gender, dtype: object

#### 소속사 column 추가 위한 작업

In [17]:
add_agency = []
add_agency = idol_list.agency
add_agency

0    서울인 프로덕션
1        쿨컴퍼니
2    더 터보 컴퍼니
3    팀 엔터테인먼트
Name: agency, dtype: object

In [19]:
test_list = [title_ary, date_ary, genre_ary, lyricist_ary, lyrics_ary]
column_names = ['년도','가수','제목','장르','작사가', '가사']
column_v = [date_ary, singer_ary, title_ary, genre_ary, lyricist_ary, lyrics_ary]

test_df = pd.DataFrame(column_v)

In [20]:
test_p = test_df.head(6).T
test_p.head()

,0,1,2,3,4,5
0,1997.04.01,룰라,3!4!,댄스,이현도,여기 숨쉬는 이 시간은 나를 어데로 데려갈까 많은 기쁨과 한숨들이 뒤섞인 이곳에서 ...
1,1995,룰라,날개 잃은 천사,댄스,이건우,아 그럴거야 나를 아낄려고 굳이 내게 말 안하고 멀리 떠나갔던가 아 그럴거야 나를 ...
2,1996.06.15,룰라,3!4!,댄스,이현도,여기 숨쉬는 이 시간은 나를 어데로 데려갈까 많은 기쁨과 한숨들이 뒤섞인 이 곳에서...
3,1995,룰라,천상유애(天上有愛),댄스,이건우,저 끝에서 끝까지 목이 메이도록 말하고 말거야 무척 소중한걸 안다고 거리마다 외칠 ...
4,1997,룰라,연인,댄스,이상민,바보 바보 바보야 바보 바보 바보야 바보 바보 바보야 사랑 앞에서 오늘도 넌 튕겨 ...


In [35]:
c_artist = []
c_artist = test_p.가수

In [36]:
tmp_artist = []
tmp_artist = idol_list.artist

In [42]:
tmp_artist

0      룰라
1       쿨
2      터보
3    알이에프
Name: artist, dtype: object

In [45]:
c_artist

0           룰라
1           룰라
2           룰라
3           룰라
4           룰라
5           룰라
6           룰라
7           룰라
8           룰라
9     쿨 (COOL)
10    쿨 (COOL)
11    쿨 (COOL)
12    쿨 (COOL)
13    쿨 (COOL)
14          터보
15          터보
16          터보
17          터보
18          터보
19          터보
20          터보
21        R.ef
22        R.ef
23        R.ef
24        R.ef
25        R.ef
26        R.ef
27        R.ef
28        R.ef
29        R.ef
30        R.ef
Name: 가수, dtype: object

In [46]:
a_gender = []

for i in range(len(c_artist)) :
    if( c_artist[i] == "룰라"):
        a_gender.append(add_gender[0])
    elif( c_artist[i] == "쿨 (COOL)" ):
        a_gender.append(add_gender[1])
    elif( c_artist[i] == "터보"):
        a_gender.append(add_gender[2])
    else :
        a_gender.append(add_gender[3])
#a_gender

In [47]:
a_gender

['mixed',
 'mixed',
 'mixed',
 'mixed',
 'mixed',
 'mixed',
 'mixed',
 'mixed',
 'mixed',
 'mixed',
 'mixed',
 'mixed',
 'mixed',
 'mixed',
 'male',
 'male',
 'male',
 'male',
 'male',
 'male',
 'male',
 'male',
 'male',
 'male',
 'male',
 'male',
 'male',
 'male',
 'male',
 'male',
 'male']

In [48]:
a_agency = []

for i in range(len(c_artist)) :
    if( c_artist[i] == "룰라"):
        a_agency.append(add_agency[0])
    elif( c_artist[i] =="쿨 (COOL)"):
        a_agency.append(add_agency[1])
    elif( c_artist[i] == "터보"):
        a_agency.append(add_agency[2])
    else:
        a_agency.append(add_agency[3])

#a_agency

In [49]:
a_agency

['서울인 프로덕션',
 '서울인 프로덕션',
 '서울인 프로덕션',
 '서울인 프로덕션',
 '서울인 프로덕션',
 '서울인 프로덕션',
 '서울인 프로덕션',
 '서울인 프로덕션',
 '서울인 프로덕션',
 '쿨컴퍼니',
 '쿨컴퍼니',
 '쿨컴퍼니',
 '쿨컴퍼니',
 '쿨컴퍼니',
 '더 터보 컴퍼니',
 '더 터보 컴퍼니',
 '더 터보 컴퍼니',
 '더 터보 컴퍼니',
 '더 터보 컴퍼니',
 '더 터보 컴퍼니',
 '더 터보 컴퍼니',
 '팀 엔터테인먼트',
 '팀 엔터테인먼트',
 '팀 엔터테인먼트',
 '팀 엔터테인먼트',
 '팀 엔터테인먼트',
 '팀 엔터테인먼트',
 '팀 엔터테인먼트',
 '팀 엔터테인먼트',
 '팀 엔터테인먼트',
 '팀 엔터테인먼트']

In [50]:
rank = []

In [51]:
column_names = ['년도','가수','제목','남녀','장르','최고순위','작사가','작곡가','소속사' ,'가사']
column_v = [date_ary, singer_ary, title_ary, a_gender, genre_ary,rank,lyricist_ary,composer_ary,a_agency, lyrics_ary]
test_df = pd.DataFrame(column_v)

In [52]:
test_p = test_df.head(10).T
test_p.head()

,0,1,2,3,4,5,6,7,8,9
0,1997.04.01,룰라,3!4!,mixed,댄스,None,이현도,이현도,서울인 프로덕션,여기 숨쉬는 이 시간은 나를 어데로 데려갈까 많은 기쁨과 한숨들이 뒤섞인 이곳에서 ...
1,1995,룰라,날개 잃은 천사,mixed,댄스,None,이건우,최준영,서울인 프로덕션,아 그럴거야 나를 아낄려고 굳이 내게 말 안하고 멀리 떠나갔던가 아 그럴거야 나를 ...
2,1996.06.15,룰라,3!4!,mixed,댄스,None,이현도,이현도,서울인 프로덕션,여기 숨쉬는 이 시간은 나를 어데로 데려갈까 많은 기쁨과 한숨들이 뒤섞인 이 곳에서...
3,1995,룰라,천상유애(天上有愛),mixed,댄스,None,이건우,The Season,서울인 프로덕션,저 끝에서 끝까지 목이 메이도록 말하고 말거야 무척 소중한걸 안다고 거리마다 외칠 ...
4,1997,룰라,연인,mixed,댄스,None,이상민,이상민,서울인 프로덕션,바보 바보 바보야 바보 바보 바보야 바보 바보 바보야 사랑 앞에서 오늘도 넌 튕겨 ...


In [53]:
test_p.columns = column_names
test_p.tail(350)

,년도,가수,제목,남녀,장르,최고순위,작사가,작곡가,소속사,가사
0,1997.04.01,룰라,3!4!,mixed,댄스,None,이현도,이현도,서울인 프로덕션,여기 숨쉬는 이 시간은 나를 어데로 데려갈까 많은 기쁨과 한숨들이 뒤섞인 이곳에서 ...
1,1995,룰라,날개 잃은 천사,mixed,댄스,None,이건우,최준영,서울인 프로덕션,아 그럴거야 나를 아낄려고 굳이 내게 말 안하고 멀리 떠나갔던가 아 그럴거야 나를 ...
2,1996.06.15,룰라,3!4!,mixed,댄스,None,이현도,이현도,서울인 프로덕션,여기 숨쉬는 이 시간은 나를 어데로 데려갈까 많은 기쁨과 한숨들이 뒤섞인 이 곳에서...
3,1995,룰라,천상유애(天上有愛),mixed,댄스,None,이건우,The Season,서울인 프로덕션,저 끝에서 끝까지 목이 메이도록 말하고 말거야 무척 소중한걸 안다고 거리마다 외칠 ...
4,1997,룰라,연인,mixed,댄스,None,이상민,이상민,서울인 프로덕션,바보 바보 바보야 바보 바보 바보야 바보 바보 바보야 사랑 앞에서 오늘도 넌 튕겨 ...
5,1995,룰라,프로와 아마츄어,mixed,댄스,None,룰라,최준영,서울인 프로덕션,나는 나는 모든걸 할수가 있어 너가 나를 사랑하게 할수도 있고 너를 감동 시킬수도 ...
6,1997.04.01,룰라,비밀은 없어,mixed,댄스,None,박선만,박선만,서울인 프로덕션,우우 정말이야 이제 그대에게 비밀은 없어 우우 알고 있지 사랑한단 말은 안해도 처음...
7,1995,룰라,사랑법,mixed,댄스,None,룰라,The Season,서울인 프로덕션,하지만 사랑이 자리잡혀 간다면 진실한 사랑이 필요하기 시작한거야 갖기 위해서 해왔던...
8,1999.02,룰라,기도 (Good),mixed,댄스,None,이상민,최준영,서울인 프로덕션,Roo'Ra deeper lover live lover live 라라따리 라리라 라...
9,1995.10.01,쿨 (COOL),슬퍼지려 하기 전에,mixed,댄스,None,최준영,최준영,쿨컴퍼니,나 가끔씩 그대 생각을 할 때마다 나 가끔 그대 생각을 할때마다 늘 가까운 듯 멀게...


In [54]:
test_p.shape

(31, 10)

In [55]:
# 곡 전처리
test_p['제목'].isnull().sum()

0

In [56]:
test_p.tail()

,년도,가수,제목,남녀,장르,최고순위,작사가,작곡가,소속사,가사
26,1997,R.ef,심연(深淵),male,댄스,None,3534,홍재선,팀 엔터테인먼트,너의 기억 깨져버려 너의 기억 지나가버려 나의 기억 기억에서 말이야 지워져 버리라는...
27,1996.04,R.ef,마음속을 걸어가,male,댄스,None,홍재선,홍재선,팀 엔터테인먼트,넌 지금 어딨는지 난 중요치 않아 누구와 함께 있는 것이 중요해 나 아닌 다른 사람...
28,1996.04,R.ef,Jazz,male,발라드,None,홍재선,홍재선,팀 엔터테인먼트,둘이서 함께 꾸는 꿈 음악도 표현 못했던 그 누구도 알 수 없는 상처 담배를 피워보...
29,1999,R.ef,찬란한 사랑,male,댄스,None,홍재선,홍재선,팀 엔터테인먼트,너에게 아무런 도움이 되지 못했어 내가 내 불행마저 감당할 수 없는데 그냥 너를 좋...
30,1998.05.01,R.ef,Never Ending Story,male,댄스,None,이승호,윤일상,팀 엔터테인먼트,여기저기 하루종일 너를 찾아 헤매어 봐도 니 모습 간데없고 추억들만 나를 반겼어 이...


In [57]:
test_p.to_csv('k_pop_now.tsv',index=False,header=None,sep='\t')